# Pre-process POP909 for Polyphemus
- split 909 into phrases instead of bars
- Phrases are of varying bar-length --> Add padding??
- Use 4/4 only

In [3]:
import os
import time
import sys
import multiprocessing
import itertools
import argparse
from itertools import product

import numpy as np
import tqdm
import pypianoroll as pproll
import muspy

In [1]:
filepath = "POP909/001/001.mid"

In [5]:
pproll_song = pproll.read(filepath, resolution=8)
muspy_song = muspy.read(filepath)

In [15]:
for i in range(1,10):
    filepath = f"POP909/00{i}/00{i}.mid"
    print(f"Processing {filepath}")
    pproll_song = pproll.read(filepath, resolution=8)
    muspy_song = muspy.read(filepath)
    # Only accept songs that have a time signature of 4/4 and no time changes
    for t in muspy_song.time_signatures:
        # print(t)
        if t.numerator != 4 or t.denominator != 4:
            print("Song skipped ({}/{} time signature)".
                  format(t.numerator, t.denominator))
        else:
            print(f"Song accepted! {filepath}")

Processing POP909/001/001.mid
Song skipped (2/4 time signature)
Processing POP909/002/002.mid
Song skipped (2/4 time signature)
Processing POP909/003/003.mid
Song skipped (2/4 time signature)
Processing POP909/004/004.mid
Song skipped (2/4 time signature)
Processing POP909/005/005.mid
Song skipped (2/4 time signature)
Processing POP909/006/006.mid
Song skipped (2/4 time signature)
Processing POP909/007/007.mid
Song skipped (2/4 time signature)
Processing POP909/008/008.mid
Song accepted! POP909/008/008.mid
Song skipped (2/4 time signature)
Processing POP909/009/009.mid
Song skipped (2/4 time signature)


In [16]:
pproll_song.tracks

[StandardTrack(name='MELODY', program=0, is_drum=False, pianoroll=array(shape=(3792, 128), dtype=uint8)),
 StandardTrack(name='BRIDGE', program=0, is_drum=False, pianoroll=array(shape=(3792, 128), dtype=uint8)),
 StandardTrack(name='PIANO', program=0, is_drum=False, pianoroll=array(shape=(3792, 128), dtype=uint8))]

### TODO:
1. Figure out how to process 2/4 time signature in midi files
2. Figure out how to split by number of bars, since we need to split phrases by length in number of bars.